In [ ]:
!git clone https://github.com/mchon89/Udemy.git

Cloning into 'Udemy'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), done.


In [ ]:
!pip install html2text memory_profiler textdistance textdistance[extras]

In [ ]:
from html2text import HTML2Text
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfTransformer
h = HTML2Text()

In [ ]:
dataset = pd.read_csv('Udemy/udemy2.csv')
dataset[10:20]

,id,title,url,is_paid,price,professor,description,num_subscribers
10,1072494,Statistics explained completely,/statistics-course/,True,$20,Enthusia Educational Hub,<p>This Statistics course will cover the impor...,3196
11,1004980,Become a Calculus Master,/learn-calculus/,True,$20,Apoorva Suryadevara,<p><strong>In Calculus you'll still be doing a...,2803
12,693138,Simply Jesus,/simply-jesus/,True,$80,N.T. Wright,<p>In this course <em>Simply Jesus</em> studen...,2106
13,1195778,Xamarin Forms: Build Android Apps for Absolute...,/xamarin-forms-for-absolute-beginners/,True,$95,Sobhan N.,<p>___________________________________________...,1222
14,342322,Chinese Made Easy L2: Understand 79% of Chines...,/dc-level-2/,True,$20,Felix Lättman,<p> \t<strong><em>LATEST: COURSE UPDATED IN A...,4519
15,830298,Scratch Game Programming for Young Adults,/scratch-game-programming/,True,$35,Al Sweigart,<p>Scratch is the best educational programming...,5028
16,466208,Paul and His Letter to the Galatians,/paul-and-his-letter-to-the-galatians/,True,$120,N.T. Wright,"<p style="""">'Being Transformed by the Renewing...",2556
17,785702,How to Make a Difference by Nicholas Kristof &...,/how-to-make-a-difference-by-nicholas-kristof-...,True,$120,Nicholas Kristof,<p>This one-of-a-kind course by Nick Kristof a...,1677
18,128298,Become a Calculus 2 Master,/calculus-2/,True,$150,Krista King,"<p><strong class=""redactor-inline-converted"">H...",5249
19,858678,The Day the Revolution Began,/the-day-the-revolution-began/,True,$80,N.T. Wright,<p>The renowned scholar Anglican bishop and be...,1175


In [ ]:
descriptions = dataset['description']

In [ ]:
X_train = []
for line in descriptions:
  line= h.handle(line)
  re.sub(r'[^\x00-\x7f]',r'', line)
  line = line.strip(' ')
  if line:
    X_train.append(line)

In [ ]:
stopwords_ = ["s","i","me",'my','myself','we','our','ours','a','the','your','ourselves','you',"you're","you've","you'll","you'd",'your', 
'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', "she","she's", 'her', 'hers', 'herself', 'it', "it's",'its', 'itself', 
'they', 'them', 'their','theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll",'these','those','am','is','are',
'was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as',
'until','while','of','at','by','for','with','about','against', 'between','into','through','during','before','after','above','below','to',
'from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all',
'any','both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 
's','t', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 
'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",'ma','mightn',
"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn',
"wouldn't",'best','different',"won't","couldn't","mustn't","didn't"]

custom_stop = ["adv", "fundamentals", "course", "advance", "beginner", "beginners","nbsp", "help", "would", "could", "courses","also","get",
"give","class","much",'copy0','copy1','nbsp','course','copy','best','different',"learn",'course','learning','learner']

stopwords_= stopwords_ + custom_stop

In [ ]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
for index,line in enumerate(X_train):
  X_train[index] = pre_process(line)

In [ ]:
X_test = []
with open('data.txt') as filename:
  for line in filename:
    line = pre_process(line)
    X_test.append(line)

In [ ]:
cv=CountVectorizer(max_df=0.8,stop_words=stopwords_,ngram_range=(1,2),analyzer='word')
word_count_vector=cv.fit_transform(X_test)

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
feature_names=cv.get_feature_names()

In [ ]:
def test_function(doc,feature_names):

  #generate tf-idf for the given document
  tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
  #sort the tf-idf vectors by descending order of scores
  sorted_items=sort_coo(tf_idf_vector.tocoo())
  #extract only the top n; n here is 10
  keywords=extract_topn_from_vector(feature_names,sorted_items,10)
  
  # # now print the results
  # print("\n=====Doc=====")
  # print(doc)
  # print("\n===Keywords===")
  # for k in keywords:
  #   print(k,keywords[k])
  return keywords

In [ ]:
!mv datasets_93858_218274_linkedin\ skill keyword.txt

In [ ]:
mapping_dictionary = {}
with open('keyword.txt') as file_object:
  for line in file_object:
    line = line.strip('\n')
    line = line.lower()
    mapping_dictionary[line[0]] = mapping_dictionary.get(line[0],[])
    mapping_dictionary[line[0]].append(line)

In [ ]:
mapping_dictionary

In [ ]:
import textdistance

In [ ]:
def compare_func(keyword,mapping_dictionary = mapping_dictionary):
  # print(keyword.items())
  if keyword:
    start_char = keyword[0]
  compare = mapping_dictionary.get(start_char,[])
  # print(compare)
  if compare:
    maximum = 0
    max_word = None
    for word in compare:
      curr = textdistance.jaro_winkler(word,keyword)
      maximum = max(maximum,curr)
      max_word = word if curr == maximum else max_word
    print("Ori: {} \t\t Mapped: {} ".format(keyword, max_word,maximum))

In [ ]:
for documents in X_test[10:25]:
  keywords = test_function(documents,feature_names)
  for lines in keywords:
    compare_func(lines)
  print('\n')


Ori: html 		 Mapped: html 
Ori: tags 		 Mapped: tags 
Ori: elements 		 Mapped: elements 
Ori: website html 		 Mapped: website translation 
Ori: html create 		 Mapped: html5 boilerplate 
Ori: content 		 Mapped: contentdm 
Ori: html tags 		 Mapped: html 
Ori: page html 		 Mapped: page layout 
Ori: markup language 		 Mapped: markup languages 
Ori: markup 		 Mapped: market updates 


Ori: java 		 Mapped: java 
Ori: advanced java 		 Mapped: advanced 
Ori: advanced 		 Mapped: advanced 
Ori: applications 		 Mapped: application 
Ori: concepts 		 Mapped: concerts 
Ori: using core 		 Mapped: unicode 
Ori: us already 		 Mapped: user analysis 
Ori: tutorial giving 		 Mapped: tutorials 
Ori: thus essential 		 Mapped: test suites 
Ori: thus 		 Mapped: tubs 


Ori: hadoop 		 Mapped: hadoop 
Ori: data 		 Mapped: datatel 
Ori: framework 		 Mapped: framework 
Ori: www knowledgehut 		 Mapped: www 
Ori: pros cons 		 Mapped: procmon 
Ori: nodes 		 Mapped: node.js 
Ori: knowledgehut com 		 Mapped: knowledge